# Cleaning unique user data

In [1]:
# importing necessary packages
import pandas as pd
import requests
from dotenv import load_dotenv
load_dotenv()
import os
import json
import flickrapi
import time
import numpy as np
import re
#import fuzzywuzzy
#from fuzzywuzzy import fuzz, process
#import pycountry

In [2]:
# read csv into df
df_user_data = pd.read_csv('data/unique_owner.csv')

/var/folders/n9/f309cdzj1vg84wrrq1h5nskm0000gn/T/ipykernel_21688/63229535.py:2: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_user_data = pd.read_csv('data/unique_owner.csv')


In [3]:
df_user_data.shape

(38801, 38)

In [4]:
# check df
df_user_data.head(10)

,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,stat,person_id,person_nsid,...,person_has_free_educational_resources,person_timezone_label,person_timezone_offset,person_timezone_timezone_id,person_timezone_timezone,person_pro_badge,person_expire,person_mbox_sha1sum__content,code,message
0,0,0.0,0.0,0.0,0.0,0.0,0.0,ok,197843899@N04,197843899@N04,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,1.0,1.0,1.0,1.0,1.0,ok,26745338@N06,26745338@N06,...,0.0,Brasilia,-03:00,America/Sao_Paulo,20.0,NaN,NaN,NaN,NaN,NaN
2,2,2.0,2.0,2.0,2.0,2.0,2.0,ok,31331368@N00,31331368@N00,...,0.0,"Canberra, Melbourne, Sydney",+10:00,Australia/Canberra,68.0,standard,0.000000e+00,NaN,NaN,NaN
3,3,3.0,3.0,3.0,3.0,3.0,3.0,ok,122687277@N03,122687277@N03,...,0.0,NaN,NaN,NaN,NaN,standard,0.000000e+00,NaN,NaN,NaN
4,4,4.0,4.0,4.0,4.0,4.0,4.0,ok,196275842@N08,196275842@N08,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,5.0,5.0,5.0,5.0,5.0,5.0,ok,44842144@N03,44842144@N03,...,0.0,NaN,NaN,NaN,NaN,standard,0.000000e+00,NaN,NaN,NaN
6,6,6.0,6.0,6.0,6.0,6.0,6.0,ok,45008553@N04,45008553@N04,...,0.0,"Bangkok, Hanoi, Jakarta",+07:00,Asia/Bangkok,55.0,NaN,NaN,NaN,NaN,NaN
7,7,7.0,7.0,7.0,7.0,7.0,7.0,ok,158117692@N02,158117692@N02,...,0.0,NaN,NaN,NaN,NaN,standard,1.614069e+09,NaN,NaN,NaN
8,8,8.0,8.0,8.0,8.0,8.0,8.0,ok,118071797@N03,118071797@N03,...,0.0,NaN,NaN,NaN,NaN,standard,0.000000e+00,NaN,NaN,NaN
9,9,9.0,9.0,9.0,9.0,9.0,9.0,ok,169491036@N06,169491036@N06,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Drop unnecessary columns

In [5]:
# check value count for is deleted
print(df_user_data.person_is_deleted.value_counts())
print(df_user_data.person_is_deleted.isna().sum())
# because there is only 1 value (False) and 24 NaN I decided to drop the column

0.0    38777
Name: person_is_deleted, dtype: int64
24


In [6]:
# list of columns to drop
columns_to_drop = ['Unnamed: 0.6','Unnamed: 0.5','Unnamed: 0.4', 'Unnamed: 0.3',
       'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'stat','person_nsid','person_is_deleted',
       'person_iconserver','person_iconfarm','person_has_stats',
       'person_description__content',
       'person_photosurl__content', 'person_profileurl__content',
       'person_mobileurl__content','person_has_adfree', 'person_has_free_standard_shipping',
       'person_has_free_educational_resources','person_pro_badge','person_mbox_sha1sum__content','message','person_expire', 'code','person_timezone_timezone']

In [7]:
# check if its the right number
len(columns_to_drop)

26

In [8]:
# drop 26 columns
df_user_data_cleaned = df_user_data.drop(columns=columns_to_drop)

In [9]:
# check columns
df_user_data_cleaned.columns

Index(['person_id', 'person_ispro', 'person_path_alias',
       'person_username__content', 'person_realname__content',
       'person_location__content', 'person_photos_firstdatetaken__content',
       'person_photos_firstdate__content', 'person_photos_count__content',
       'person_timezone_label', 'person_timezone_offset',
       'person_timezone_timezone_id'],
      dtype='object')

## Alter column names

In [10]:
# replace 'person' with user and delete '_content'
df_user_data_cleaned.columns = df_user_data_cleaned.columns.str.replace('person','user').str.replace('__content','')


In [11]:
df_user_data_cleaned.columns

Index(['user_id', 'user_ispro', 'user_path_alias', 'user_username',
       'user_realname', 'user_location', 'user_photos_firstdatetaken',
       'user_photos_firstdate', 'user_photos_count', 'user_timezone_label',
       'user_timezone_offset', 'user_timezone_timezone_id'],
      dtype='object')

## delete NaN from user_location, bc it's our most important column

In [12]:
# clear NaN values from user_locations
df_user_data_cleaned = df_user_data_cleaned.dropna(subset='user_location').reset_index(drop=True)
#df_user_data_cleaned = df_user_data_cleaned.reset_index(drop=True)

In [13]:
# check how much is still there
df_user_data_cleaned.shape

(14469, 12)

In [14]:
df_user_data_cleaned.head()

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id
0,26745338@N06,0.0,danisonksen,dani.sonksen,Daniela Sönksen,"Melbourne, Australia",2005-05-09 10:56:54,1.212696e+09,850.0,Brasilia,-03:00,America/Sao_Paulo
1,31331368@N00,1.0,lonelyradio,lonely radio,andrew j. cosgriff,"Melbourne, Australia",2000-02-01 00:00:00,1.099811e+09,13381.0,"Canberra, Melbourne, Sydney",+10:00,Australia/Canberra
2,122687277@N03,1.0,philipmallis,philip.mallis,Philip Mallis,Melbourne,2007-09-21 04:37:37,1.397393e+09,10793.0,NaN,NaN,NaN
3,45008553@N04,0.0,reaksmeyyean,REAKSMEY Yean – GEORGE,REAKSMEY Yean – GEORGE,"Phnom Penh, Kingdom of Cambodia",2005-11-04 20:32:27,1.259290e+09,236.0,"Bangkok, Hanoi, Jakarta",+07:00,Asia/Bangkok
4,158117692@N02,1.0,NaN,Julie McLennan,Julie McLennan,"Melbourne, Australia",2018-01-08 22:23:51,1.515479e+09,2533.0,NaN,NaN,NaN


In [15]:
len(df_user_data_cleaned)

14469

In [16]:
# check percentage of leftover data - 37% left data
x = (len(df_user_data_cleaned)/len(df_user_data))*100
print(x)

37.29027602381382


## alter dtypes

In [17]:
# check dtypes
df_user_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14469 entries, 0 to 14468
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   user_id                     14469 non-null  object 
 1   user_ispro                  14469 non-null  float64
 2   user_path_alias             9890 non-null   object 
 3   user_username               14469 non-null  object 
 4   user_realname               12591 non-null  object 
 5   user_location               14469 non-null  object 
 6   user_photos_firstdatetaken  14468 non-null  object 
 7   user_photos_firstdate       14468 non-null  float64
 8   user_photos_count           14469 non-null  float64
 9   user_timezone_label         13283 non-null  object 
 10  user_timezone_offset        13283 non-null  object 
 11  user_timezone_timezone_id   13283 non-null  object 
dtypes: float64(3), object(9)
memory usage: 1.3+ MB


In [18]:
df_user_data_cleaned.user_id

0         26745338@N06
1         31331368@N00
2        122687277@N03
3         45008553@N04
4        158117692@N02
             ...      
14464    131418876@N03
14465     42905557@N00
14466     16939187@N06
14467     97454045@N07
14468     35939355@N04
Name: user_id, Length: 14469, dtype: object

In [19]:
# user_ispro to bool
df_user_data_cleaned.user_ispro=df_user_data_cleaned.user_ispro.astype('bool')

In [20]:
df_user_data_cleaned[['user_timezone_offset','user_timezone_timezone_id','user_timezone_label']].sample(5)

,user_timezone_offset,user_timezone_timezone_id,user_timezone_label
9576,-08:00,PST8PDT,Pacific Time (US & Canada); Tijuana
1306,+01:00,Europe/Brussels,"Brussels, Copenhagen, Madrid, Paris"
8542,NaN,NaN,NaN
2093,+00:00,Europe/London,"GMT: Dublin, Edinburgh, Lisbon, London"
1454,+02:00,Europe/Helsinki,"Helsinki, Kyiv, Riga, Sofia, Tallinn, Vilnius"


In [21]:
# user_location','user_realname','user_username','user_path_alias,... to string except user_id, bc it needs to be original to query the API
df_user_data_cleaned[['user_location','user_realname','user_username','user_path_alias','user_timezone_offset','user_timezone_timezone_id','user_timezone_label']] = df_user_data_cleaned[['user_location','user_realname','user_username','user_path_alias','user_timezone_offset','user_timezone_timezone_id','user_timezone_label']].astype('string')

In [22]:

df_user_data_cleaned = df_user_data_cleaned.apply(lambda x: x.str.lower() if x.dtype == "string" else x) 

In [23]:
# user_id to string
df_user_data_cleaned.user_id=df_user_data_cleaned.user_id.astype('string')

In [25]:
df_user_data_cleaned.sample(19)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id
11858,18496927@N00,True,cthonus,cthonus,<NA>,"edinburgh, uk",1985-06-01 12:00:00,1.157875e+09,31884.0,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london
12143,59621031@N03,False,minispirit,minispirit,john yin,"adelaide, australia",2011-01-10 19:44:02,1.297951e+09,2612.0,adelaide,+09:30,australia/adelaide
11347,37742647@N06,False,vaninaparras,vaninaparras,vanina parras,"east bay, california",2007-04-20 14:51:21,1.240546e+09,5407.0,pacific time (us & canada); tijuana,-08:00,pst8pdt
12692,143408826@N08,True,<NA>,sven bonorden,sven bonorden,"dortmund, germany",2009-01-10 12:15:56,1.472909e+09,934.0,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam
5768,153660805@N05,False,<NA>,cormack owle,cormack owle,united states,2019-11-14 19:47:28,1.573779e+09,331.0,eastern time (us & canada),-05:00,est5edt
1766,67731822@N00,False,nathanaelarcher,(nathanael.archer),jean-françois bodart,"cuincy, france",2002-09-24 01:14:21,1.099045e+09,3312.0,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london
1120,14730981@N08,True,<NA>,james e. petts,james petts,"london, england",1985-01-01 00:00:00,1.191757e+09,23499.0,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london
5267,28045633@N00,True,h_duncan,howard j duncan,howard duncan,"victoria bc, canada",2001-02-01 17:12:01,1.183343e+09,4784.0,pacific time (us & canada); tijuana,-08:00,pst8pdt
7725,39840595@N04,True,andyrees1,andyrees1,andrew rees,"cardiff, wales",0000-00-00 00:00:00,1.327237e+09,1640.0,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london
5658,69421573@N08,True,<NA>,ken mattison,ken mattison,"greendale, wisconsin, usa",2012-02-24 09:10:04,1.341447e+09,1787.0,central time (us & canada),-06:00,cst6cdt


In [24]:
df_user_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14469 entries, 0 to 14468
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   user_id                     14469 non-null  string 
 1   user_ispro                  14469 non-null  bool   
 2   user_path_alias             9890 non-null   string 
 3   user_username               14469 non-null  string 
 4   user_realname               12591 non-null  string 
 5   user_location               14469 non-null  string 
 6   user_photos_firstdatetaken  14468 non-null  object 
 7   user_photos_firstdate       14468 non-null  float64
 8   user_photos_count           14469 non-null  float64
 9   user_timezone_label         13283 non-null  string 
 10  user_timezone_offset        13283 non-null  string 
 11  user_timezone_timezone_id   13283 non-null  string 
dtypes: bool(1), float64(2), object(1), string(8)
memory usage: 1.2+ MB


In [26]:
# user_photos_count to int
df_user_data_cleaned.user_photos_count = df_user_data_cleaned.user_photos_count.astype('int')

In [ ]:
df_user_data_cleaned.user_photos_firstdate

In [27]:
# user_firstdate unix to datetime
df_user_data_cleaned.user_photos_firstdate = pd.to_datetime(df_user_data_cleaned.user_photos_firstdate,unit='s',errors='coerce')

In [ ]:
# check type
type(df_user_data_cleaned.user_photos_firstdatetaken)

In [28]:
# alter to string
df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.astype('string')

In [ ]:
df_user_data_cleaned.user_photos_firstdatetaken[9:15]

In [29]:
df_user_data_cleaned.user_photos_firstdatetaken = pd.to_datetime(df_user_data_cleaned.user_photos_firstdatetaken, dayfirst=True,format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [ ]:
# because coerce replaces error values with NaT 
# firstdatetaken is mysql datetime that starts with 1000-00-00 00:00:00 we "lost" 787 values
df_user_data_cleaned.user_photos_firstdatetaken.count()

In [ ]:
df_user_data_cleaned.info()

## check for duplicates  

In [ ]:
df_user_data_cleaned.duplicated().value_counts()

In [ ]:
# we find more duplicates for one column, otherwise it compares the row
df_user_data_cleaned.duplicated(subset='user_id').value_counts()

In [ ]:
# look at duplicates
df_user_data_cleaned[df_user_data_cleaned.duplicated(keep=False)].sort_values(by='user_id')

## keep existing user id's in user_in_df for updates

In [30]:
# drop duplicates and store existing user ids to a list
user_in_df=[]
df_user_data_cleaned = df_user_data_cleaned.drop_duplicates(subset='user_id')

for i in df_user_data_cleaned.user_id.unique():
    user_in_df.append(i)

In [ ]:
df_user_data_cleaned.shape

In [ ]:
# check if everything was right
print(user_in_df)
print(len(user_in_df))
# check type
type(user_in_df)

In [ ]:
df_user_data_cleaned.sample(5)

## split user location into useful columns

In [ ]:
# check format of user_location
df_user_data_cleaned.user_location

In [ ]:
## conclusion timestamp: as I figured out flickr is creating timezone columns with arguments of location, if they fit in at least on 'city'and/or 'country' format
# check how many values for timezone_offset
df_user_data_cleaned.user_timezone_offset.isna().sum()

In [ ]:
# note there is a wrong TZ
df_user_data_cleaned.loc[7152]

In [31]:
df_temp = df_user_data_cleaned.user_location.str.split(', ',expand=True)

In [32]:
df_temp

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,melbourne,australia,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,melbourne,australia,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,melbourne,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,phnom penh,kingdom of cambodia,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,melbourne,australia,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14464,alcalá de henares,spain,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
14465,padova,italy,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
14466,cagliari,italia,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
14467,szeged,hungary,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [39]:
df_temp.columns

Index(['0', '1', '2'], dtype='string')

In [34]:
df_temp.columns = df_temp.columns.astype('string')

In [37]:
columns_keep_temp = ['0','1','2']

In [38]:
df_temp = df_temp.loc[:, columns_keep_temp]

In [ ]:
# "rename" columns from range to names
#df_temp.columns=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21']

In [40]:
# merge new location columns to dataframe
df_user_data_cleaned=pd.merge(df_user_data_cleaned,df_temp,left_index=True, right_index=True)

In [41]:
df_user_data_cleaned.head()

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2
0,26745338@N06,False,danisonksen,dani.sonksen,daniela sönksen,"melbourne, australia",2005-05-09 10:56:54,2008-06-05 20:03:04,850,brasilia,-03:00,america/sao_paulo,melbourne,australia,<NA>
1,31331368@N00,True,lonelyradio,lonely radio,andrew j. cosgriff,"melbourne, australia",2000-02-01 00:00:00,2004-11-07 06:59:09,13381,"canberra, melbourne, sydney",+10:00,australia/canberra,melbourne,australia,<NA>
2,122687277@N03,True,philipmallis,philip.mallis,philip mallis,melbourne,2007-09-21 04:37:37,2014-04-13 12:48:29,10793,<NA>,<NA>,<NA>,melbourne,<NA>,<NA>
3,45008553@N04,False,reaksmeyyean,reaksmey yean – george,reaksmey yean – george,"phnom penh, kingdom of cambodia",2005-11-04 20:32:27,2009-11-27 02:43:31,236,"bangkok, hanoi, jakarta",+07:00,asia/bangkok,phnom penh,kingdom of cambodia,<NA>
4,158117692@N02,True,<NA>,julie mclennan,julie mclennan,"melbourne, australia",2018-01-08 22:23:51,2018-01-09 06:23:51,2533,<NA>,<NA>,<NA>,melbourne,australia,<NA>


In [ ]:
# only the first 3 columns seem to have reliable data, so I dropped the rest
#df_user_data_cleaned = df_user_data_cleaned.drop(columns=['3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21'])

In [ ]:
df_user_data_cleaned.columns

In [ ]:
df_user_data_cleaned.shape

In [ ]:
df_user_data_cleaned.head()

In [ ]:
# check if there is a timezone generated with only usa
df_user_data_cleaned.loc[6316]

## figuring out what happens in the location column

In [ ]:
# change dt to list
# split without expand=true
df_split = df_user_data_cleaned.user_location.str.split(', ')
df_split.tolist()

### decided to only keep the first 3 columns as they fit the format

In [ ]:
three_list=[]
for x in df_split:
    if len(x)<=3:
        three_list.append(x)
print(len(three_list))

In [ ]:
rest_list=[]
for x in df_split:
    if len(x)>=4:
        rest_list.append(x)
print(len(rest_list))

In [ ]:
one_list=[]
for x in df_split:
    if len(x)<=1:
        one_list.append(x)
print(len(one_list))

In [ ]:
len(df_user_data_cleaned.user_location)

In [ ]:
# 94% follow the format
(len(rest_list)/len(three_list))*100

## getting new df's country,city, state

In [42]:
df_world_countries = pd.read_csv('data/world.csv')

In [ ]:
df_world_countries.head(2)

In [43]:
df_world_countries.drop(columns='id')

,alpha2,alpha3,ar,bg,cs,da,de,el,en,eo,...,ro,ru,sk,sl,sr,sv,th,uk,zh,zh-tw
0,af,afg,أفغانستان,Афганистан,Afghánistán,Afghanistan,Afghanistan,Αφγανιστάν,Afghanistan,Afganio,...,Afganistan,Афганистан,Afganistan,Afganistan,Avganistan,Afghanistan,อัฟกานิสถาน,Афганістан,阿富汗,阿富汗
1,ax,ala,جزر أولاند,Оландски острови,Alandy,Ålandsøerne,Åland,Ώλαντ,Åland Islands,Alando,...,Insulele Åland,Аландские острова,Alandy,Ålandski otoki,Olandska Ostrva,Åland,หมู่เกาะโอลันด์,Аландські острови,奥兰,奧蘭
2,al,alb,ألبانيا,Албания,Albánie,Albanien,Albanien,Αλβανία,Albania,Albanio,...,Albania,Албания,Albánsko,Albanija,Albanija,Albanien,แอลเบเนีย,Албанія,阿尔巴尼亚,阿爾巴尼亞
3,dz,dza,الجزائر,Алжир,Alžírsko,Algeriet,Algerien,Αλγερία,Algeria,Alĝerio,...,Algeria,Алжир,Alžírsko,Alžirija,Alžir,Algeriet,แอลจีเรีย,Алжир,阿尔及利亚,阿爾及利亞
4,as,asm,ساموا الأمريكية,Американска Самоа,Americká Samoa,Amerikansk Samoa,Amerikanisch-Samoa,Αμερικανική Σαμόα,American Samoa,Usona Samoo,...,Samoa americană,Американское Самоа,Americká Samoa,Ameriška Samoa,Američka Samoa,Amerikanska Samoa,อเมริกันซามัว,Американське Самоа,美属萨摩亚,美屬薩摩亞
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,wf,wlf,واليس وفوتونا,Уолис и Футуна,Wallis a Futuna,Wallis og Futuna,Wallis und Futuna,Ουαλίς και Φουτουνά,Wallis and Futuna,Valiso kaj Futuno,...,Wallis și Futuna,Уоллис и Футуна,Wallis a Futuna,Wallis in Futuna,Valis i Futuna,Wallis- och Futunaöarna,วอลิสและฟูตูนา,Волліс і Футуна,瓦利斯和富图纳,瓦利斯和富圖那
245,eh,esh,الصحراء الغربية,Западна Сахара,Západní Sahara,Vestsahara,Westsahara,Δυτική Σαχάρα,Western Sahara,Okcidenta Saharo,...,Sahara Occidentală,САДР,Západná Sahara,Zahodna Sahara,Zapadna Sahara,Västsahara,เวสเทิร์นสะฮารา,Західна Сахара,西撒哈拉,西撒哈拉
246,ye,yem,اليمن,Йемен,Jemen,Yemen,Jemen,Υεμένη,Yemen,Jemeno,...,Yemen,Йемен,Jemen,Jemen,Jemen,Jemen,เยเมน,Ємен,也门,葉門
247,zm,zmb,زامبيا,Замбия,Zambie,Zambia,Sambia,Ζάμπια,Zambia,Zambio,...,Zambia,Замбия,Zambia,Zambija,Zambija,Zambia,แซมเบีย,Замбія,赞比亚,尚比亞


In [ ]:
df_world_countries.columns

In [44]:
#creating a second column en
df_world_countries['en2']= df_world_countries['en']

In [ ]:
# getting countries for laguages flickr has on the website (en,de,es,fr,zh,ko,it,pt,ru)

# creating dictionaries for each language

In [46]:
dict_country_ar= df_world_countries.set_index('ar')['en'].to_dict()
dict_country_bg= df_world_countries.set_index('bg')['en'].to_dict()
dict_country_cs= df_world_countries.set_index('cs')['en'].to_dict()
dict_country_da= df_world_countries.set_index('da')['en'].to_dict()
dict_country_de= df_world_countries.set_index('de')['en'].to_dict()
dict_country_el= df_world_countries.set_index('el')['en'].to_dict()
dict_country_en= df_world_countries.set_index('en')['en2'].to_dict()
dict_country_eo= df_world_countries.set_index('eo')['en'].to_dict()
dict_country_es= df_world_countries.set_index('es')['en'].to_dict()
dict_country_et= df_world_countries.set_index('et')['en'].to_dict()
dict_country_eu= df_world_countries.set_index('eu')['en'].to_dict()
dict_country_fi= df_world_countries.set_index('fi')['en'].to_dict()
dict_country_fr= df_world_countries.set_index('fr')['en'].to_dict()
dict_country_hr= df_world_countries.set_index('hr')['en'].to_dict()
dict_country_hu= df_world_countries.set_index('hu')['en'].to_dict()
dict_country_hy= df_world_countries.set_index('hy')['en'].to_dict()
dict_country_it= df_world_countries.set_index('it')['en'].to_dict()
dict_country_ja= df_world_countries.set_index('ja')['en'].to_dict()
dict_country_ko= df_world_countries.set_index('ko')['en'].to_dict()
dict_country_lt= df_world_countries.set_index('lt')['en'].to_dict()
dict_country_nl= df_world_countries.set_index('nl')['en'].to_dict()
dict_country_no= df_world_countries.set_index('no')['en'].to_dict()
dict_country_pl= df_world_countries.set_index('pl')['en'].to_dict()
dict_country_pt= df_world_countries.set_index('pt')['en'].to_dict()
dict_country_ro= df_world_countries.set_index('ro')['en'].to_dict()
dict_country_ru= df_world_countries.set_index('ru')['en'].to_dict()
dict_country_sk= df_world_countries.set_index('sk')['en'].to_dict()
dict_country_sl= df_world_countries.set_index('sl')['en'].to_dict()
dict_country_sr= df_world_countries.set_index('sr')['en'].to_dict()
dict_country_sv= df_world_countries.set_index('sv')['en'].to_dict()
dict_country_th= df_world_countries.set_index('th')['en'].to_dict()
dict_country_uk= df_world_countries.set_index('uk')['en'].to_dict()
dict_country_zh= df_world_countries.set_index('zh')['en'].to_dict()
dict_country_zh_tw= df_world_countries.set_index('zh-tw')['en'].to_dict()
dict_country_alpha2= df_world_countries.set_index('alpha2')['en'].to_dict()
dict_country_alpha3= df_world_countries.set_index('alpha3')['en'].to_dict()


In [47]:
# add all dictionaries to a list
list_world_countries = [dict_country_de,dict_country_ar,dict_country_bg,dict_country_cs,dict_country_da,dict_country_el,dict_country_en,dict_country_eo,dict_country_es,dict_country_et,dict_country_eu,dict_country_fi,dict_country_fr,dict_country_hr,dict_country_hu,dict_country_hy,dict_country_ja,dict_country_lt,dict_country_nl,dict_country_no,dict_country_pl,dict_country_pt,dict_country_ro,dict_country_sk,dict_country_sl,dict_country_sr,dict_country_sv,dict_country_th,dict_country_uk,dict_country_zh_tw,dict_country_zh,dict_country_ko,dict_country_it,dict_country_ru,dict_country_alpha2,dict_country_alpha3]
len(list_world_countries)

36

In [99]:
dict_country_code = df_world_countries.set_index('en')['alpha2'].to_dict()

In [106]:
dict_country_code = {k.lower(): v.lower() for k, v in dict_country_code.items()}

In [114]:
dict_country_code ['england'] = 'gb'
dict_country_code ['scotland'] = 'gb'
dict_country_code ['united states'] = 'us'
dict_country_code ['uk'] = 'gb'
dict_country_code ['wales'] = 'gb'
dict_country_code ['united kingdom'] = 'gb'
dict_country_code ['ireland'] = 'gb'
dict_country_code ['northern ireland'] = 'gb'

In [116]:
country_code_dict_json = json.dumps(dict_country_code,indent=1)
with open("data/country_code_en_dict.json", "w") as outfile:
    outfile.write(country_code_dict_json)

In [48]:
df_world_countries[df_world_countries.alpha2 == 'cz']

,id,alpha2,alpha3,ar,bg,cs,da,de,el,en,...,ru,sk,sl,sr,sv,th,uk,zh,zh-tw,en2
59,203,cz,cze,جمهورية التشيك,Чехия,Česko,Tjekkiet,Tschechien,Τσεχία,Czechia,...,Чехия,Česko,Češka,Češka,Tjeckien,เช็กเกีย,Чехія,捷克,捷克,Czechia


In [ ]:
list_world_countries


In [ ]:
df_user_data_cleaned.sample(5)

In [49]:
#replacing NaN with zero, bc Na is the country code for Namibia
df_user_data_cleaned['0'] = df_user_data_cleaned['0'].replace(np.nan,'0')
df_user_data_cleaned['1'] = df_user_data_cleaned['1'].replace(np.nan,'0')
df_user_data_cleaned['2'] = df_user_data_cleaned['2'].replace(np.nan,'0')

In [50]:
# combining the list of dict to one dictionary
combined_dict = {key:value for d in list_world_countries for key, value in d.items()}

In [51]:
type(combined_dict)

dict

In [52]:
# lower keys and values with dict comprehension
combined_dict = {k.lower(): v.lower() for k, v in combined_dict.items()}


In [ ]:
combined_dict

In [54]:
# add edge cases to dict
combined_dict ['england'] = 'england'
combined_dict ['scotland'] = 'scotland'
combined_dict ['united states'] = 'united states of america'
combined_dict ['kingdom of cambodia'] = 'cambodia'
combined_dict ['uk'] = 'united kingdom'
combined_dict ['wales'] = 'wales'
combined_dict ['united kingdom'] = 'united kingdom'
combined_dict ['czech republic'] = 'czechia'

In [55]:
# clean columns to find more matching values
df_user_data_cleaned['0'] = df_user_data_cleaned['0'].str.lower().str.strip()
df_user_data_cleaned['1'] = df_user_data_cleaned['1'].str.lower().str.strip()
df_user_data_cleaned['2'] = df_user_data_cleaned['2'].str.lower().str.strip()

In [57]:
# creating a new column empty column with NaN's
df_user_data_cleaned['user_country']= np.nan
df_user_data_cleaned['user_country_code'] = np.nan


In [ ]:
df_user_data_cleaned.sample(5)

## iterating vice versa, bc otherwise shortcuts for states will be recognized as countries

In [58]:
# iterating over every row and dict to find fitting countries
for index, row in df_user_data_cleaned.iterrows():
    zero = row['2']
    if pd.isna(row['user_country']):
        if zero in combined_dict:
            df_user_data_cleaned.at[index, 'user_country']= combined_dict[zero] #fuzzywuzzy .apply

In [59]:
# iterating over every row and dict to find fitting countries
for index, row in df_user_data_cleaned.iterrows():
    zero = row['1']
    if pd.isna(row['user_country']):
        if zero in combined_dict:
            df_user_data_cleaned.at[index, 'user_country']= combined_dict[zero] #fuzzywuzzy .apply

In [60]:
# iterating over every row and dict to find fitting countries
for index, row in df_user_data_cleaned.iterrows():
    zero = row['0']
    if pd.isna(row['user_country']):
        if zero in combined_dict:
            df_user_data_cleaned.at[index, 'user_country']= combined_dict[zero] #fuzzywuzzy .apply


In [117]:
# looping over countries to get alpha 2 in a new column
for index, row in df_user_data_cleaned.iterrows():
    zero = row['user_country']
    if pd.isna(row['user_country_code']):
        if zero in dict_country_code:
            df_user_data_cleaned.at[index, 'user_country_code']= dict_country_code[zero]

In [108]:
dict_country_code

{'afghanistan': 'af',
 'åland islands': 'ax',
 'albania': 'al',
 'algeria': 'dz',
 'american samoa': 'as',
 'andorra': 'ad',
 'angola': 'ao',
 'anguilla': 'ai',
 'antarctica': 'aq',
 'antigua and barbuda': 'ag',
 'argentina': 'ar',
 'armenia': 'am',
 'aruba': 'aw',
 'australia': 'au',
 'austria': 'at',
 'azerbaijan': 'az',
 'bahamas': 'bs',
 'bahrain': 'bh',
 'bangladesh': 'bd',
 'barbados': 'bb',
 'belarus': 'by',
 'belgium': 'be',
 'belize': 'bz',
 'benin': 'bj',
 'bermuda': 'bm',
 'bhutan': 'bt',
 'bolivia (plurinational state of)': 'bo',
 'bonaire, sint eustatius and saba': 'bq',
 'bosnia and herzegovina': 'ba',
 'botswana': 'bw',
 'bouvet island': 'bv',
 'brazil': 'br',
 'british indian ocean territory': 'io',
 'brunei darussalam': 'bn',
 'bulgaria': 'bg',
 'burkina faso': 'bf',
 'burundi': 'bi',
 'cambodia': 'kh',
 'cameroon': 'cm',
 'canada': 'ca',
 'cabo verde': 'cv',
 'cayman islands': 'ky',
 'central african republic': 'cf',
 'chad': 'td',
 'chile': 'cl',
 'china': 'cn',
 'ch

In [118]:
df_user_data_cleaned.head(25)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code
0,26745338@N06,False,danisonksen,dani.sonksen,daniela sönksen,"melbourne, australia",2005-05-09 10:56:54,2008-06-05 20:03:04,850,brasilia,-03:00,america/sao_paulo,melbourne,australia,0,australia,au
1,31331368@N00,True,lonelyradio,lonely radio,andrew j. cosgriff,"melbourne, australia",2000-02-01 00:00:00,2004-11-07 06:59:09,13381,"canberra, melbourne, sydney",+10:00,australia/canberra,melbourne,australia,0,australia,au
2,122687277@N03,True,philipmallis,philip.mallis,philip mallis,melbourne,2007-09-21 04:37:37,2014-04-13 12:48:29,10793,<NA>,<NA>,<NA>,melbourne,0,0,NaN,NaN
3,45008553@N04,False,reaksmeyyean,reaksmey yean – george,reaksmey yean – george,"phnom penh, kingdom of cambodia",2005-11-04 20:32:27,2009-11-27 02:43:31,236,"bangkok, hanoi, jakarta",+07:00,asia/bangkok,phnom penh,kingdom of cambodia,0,cambodia,kh
4,158117692@N02,True,<NA>,julie mclennan,julie mclennan,"melbourne, australia",2018-01-08 22:23:51,2018-01-09 06:23:51,2533,<NA>,<NA>,<NA>,melbourne,australia,0,australia,au
5,169491036@N06,False,<NA>,sanzidrahmankhantamim,sanzid rahman khan tamim,"barishal, bangladesh",2015-03-24 23:04:05,2019-04-06 13:35:11,407,<NA>,<NA>,<NA>,barishal,bangladesh,0,bangladesh,bd
6,153546415@N07,True,free_aza_bird,free_aza_bird,karl baker,"melbourne, australia",2015-06-22 08:50:59,2017-04-03 01:24:16,2674,"canberra, melbourne, sydney",+10:00,australia/canberra,melbourne,australia,0,australia,au
7,78379788@N06,True,<NA>,rob lee photography,rob lee,australia,1985-01-01 00:00:00,2014-08-20 06:35:22,2867,"canberra, melbourne, sydney",+10:00,australia/canberra,australia,0,0,australia,au
8,115521748@N08,False,<NA>,footeefok,tee fok foo,"singapore, singapore",2008-08-24 12:38:45,2014-01-25 15:18:44,20678,"kuala lumpur, singapore",+08:00,asia/kuala_lumpur,singapore,singapore,0,singapore,sg
9,44340545@N05,True,gmpolice1,greater manchester police,greater manchester police,"manchester, greater manchester",1840-01-01 00:00:00,2009-11-05 13:53:07,27764,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,manchester,greater manchester,0,NaN,NaN


In [ ]:
combined_dict['kingdom of cambodia']

### fuzzywuzzy function

In [ ]:
# def fuzzy_match_country(text, choices):
#     # Use fuzzywuzzy to find the best match
#     best_match, score = process.extractOne(text, choices, scorer=fuzz.ratio)
#     # Set a threshold for a minimum score to consider a match
#     threshold = 80  # You can adjust this threshold as needed
#     if score >= threshold:
#         return best_match
#     else:
#         return None

In [ ]:
# Clean up the 'country_code_name' column
#df_user_data_cleaned['user_country_2'] = df_user_data_cleaned['1'].apply(lambda x: fuzzy_match_country(x, combined_dict.keys()))

In [ ]:
#df_user_data_cleaned['user_country_2'].value_counts()

In [ ]:
df_user_data_cleaned.sample(50)

### check edge cases

In [ ]:
df_user_data_cleaned.loc[12714]

In [ ]:
df_user_data_cleaned.loc[12247]

In [ ]:
df_user_data_cleaned.loc[657]

In [ ]:
combined_dict['cz']

In [ ]:
# changing czechia to czech republic ???
#df_user_data_cleaned.user_country.replace('czechia','czech republic', inplace=True)

In [ ]:
# df_user_data_cleaned['user_country_5'] = df_user_data_cleaned['1'].apply(lambda x: fuzzy_match_country(x, countries_dict_na_na.keys()))

In [98]:
# storing dict to json for function usage
country_dict_json = json.dumps(combined_dict,indent=1)
with open("data/country_dict_all_lang.json", "w") as outfile:
    outfile.write(country_dict_json)

## cities

In [67]:
df_world_cities = pd.read_csv('data/geonames-all-cities-with-a-population-1000.csv', delimiter=';')

In [68]:
df_world_cities.sample(8)

,Geoname ID,Name,ASCII Name,Alternate Names,Feature Class,Feature Code,Country Code,Country name EN,Country Code 2,Admin1 Code,Admin2 Code,Admin3 Code,Admin4 Code,Population,Elevation,DIgital Elevation Model,Timezone,Modification date,LABEL EN,Coordinates
51359,1271444,Gawān,Gawan,"Gawan,Gawān",P,PPL,IN,India,NaN,36,149,NaN,NaN,8744,NaN,191,Asia/Kolkata,2015-08-07,India,"28.41969, 78.35186"
98246,2997559,Lormes,Lormes,"Lorm,Lorma,Lormes,luo er mei,Лорм,洛尔梅",P,PPL,FR,France,NaN,27,58,581,58145,1389,NaN,424,Europe/Paris,2019-03-26,France,"47.28964, 3.81714"
55473,496934,Serdobsk,Serdobsk,"Bol'shaya Serdoba,Bol’shaya Serdoba,Serdobsk,S...",P,PPL,RU,Russian Federation,NaN,57,NaN,NaN,NaN,36652,NaN,191,Europe/Moscow,2019-09-05,Russian Federation,"52.45861, 44.21689"
65434,1434016,‘Alāqahdārī Saṟōbī,'Alaqahdari Sarobi,"Alakadari-Sarobi,Sarobi,Saṟōbī,`Alaqadari Saro...",P,PPLA2,AF,Afghanistan,NaN,29,2908,NaN,NaN,0,NaN,2400,Asia/Kabul,2020-06-11,Afghanistan,"32.79331, 69.0866"
122029,667227,Sighetu Marmaţiei,Sighetu Marmatiei,"Maramaros Szighet,Maramarossziget,Marmaros Sih...",P,PPLA2,RO,Romania,NaN,25,106559,NaN,NaN,40653,NaN,270,Europe/Bucharest,2020-11-11,Romania,"47.93195, 23.88603"
110736,2981178,Saint-Christol,Saint-Christol,Saint-Christol,P,PPL,FR,France,NaN,76,34,343,34246,1367,NaN,70,Europe/Paris,2020-06-10,France,"43.72806, 4.07998"
44773,3167954,San Miniato Basso,San Miniato Basso,"San Miniato,San Miniato Basso,San Miniatu,サン・ミ...",P,PPL,IT,Italy,NaN,16,PI,050032,NaN,26365,NaN,28,Europe/Rome,2015-08-07,Italy,"43.69332, 10.84119"
79704,6534510,Oltressenda Alta,Oltressenda Alta,NaN,P,PPLA3,IT,Italy,NaN,09,BG,016147,NaN,194,NaN,735,Europe/Rome,2012-02-15,Italy,"45.91519, 9.94486"


In [111]:
# clean
df_world_cities.columns = df_world_cities.columns.str.lower().str.replace(' ', '_')


In [70]:
city_columns_to_keep = ['name','ascii_name', 'alternate_names']

In [71]:
df_world_cities_split = df_world_cities.loc[:,city_columns_to_keep]

In [72]:
df_world_cities_split

,name,ascii_name,alternate_names
0,Lyudinovo,Lyudinovo,"Ljudinovo,Lyudinovo,Людиново"
1,Lubyany,Lubyany,"Lubjany,Lubyany,Лубяны"
2,Lobanovo,Lobanovo,"Lobanovo,Лобаново"
3,Linda,Linda,"Linda,Линда"
4,Leskolovo,Leskolovo,"Leskolovo,Lieskula,Лесколово"
...,...,...,...
141116,Limeira,Limeira,"Limeira,Limejra,QGB,li mei la,li mey ra,limaye..."
141117,Itaúna,Itauna,"Itauna,ita'una,yi ta wu na,Итауна,Իտաունա,ইটাউ..."
141118,Itaporanga,Itaporanga,NaN
141119,Itapicuru,Itapicuru,"Missao,Missão"


In [73]:
df_world_cities_split.name = df_world_cities_split.name.str.strip().str.lower()
df_world_cities_split.ascii_name = df_world_cities_split.ascii_name.str.strip().str.lower()
df_world_cities_split.alternate_names = df_world_cities_split.alternate_names.str.strip().str.lower()

In [74]:
df_alternate_names = df_world_cities_split.alternate_names.str.split(',', expand=True)

In [75]:
df_alternate_names.sample(8)

,0,1,2,3,4,5,6,7,8,9,...,259,260,261,262,263,264,265,266,267,268
96684,penrith,penrisu,პენრითი,ペンリス,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
81215,saulia,şăulia,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
76067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92688,ar rustaq,ar rustāq,mnh,qal`at kasra,qal‘at kasrá,rostak,rustaka,rustaq,rustāq,alrstaq,...,None,None,None,None,None,None,None,None,None,None
101623,council oaks,dandenai,darbinel,dardanel',dardanell,dardanelle,dardi,dardonnie,derdonnai,dosduail,...,None,None,None,None,None,None,None,None,None,None
64461,chuan xi,川西,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
13935,aydinlar,aydınlar,haciarap,haciharapkonagi,hacıarap,hacıharapkonağı,musar,muşar,None,None,...,None,None,None,None,None,None,None,None,None,None
40017,nag's head,nags head,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [76]:
df_alternate_names.columns = df_alternate_names.columns.astype('string')

In [77]:
columns_keep = ['0','1','2','3','4','5','6']

In [78]:
df_alternate_names = df_alternate_names.loc[:,columns_keep]

In [82]:
df_alternate_names

,0,1,2,3,4,5,6
0,ljudinovo,lyudinovo,людиново,None,None,None,None
1,lubjany,lubyany,лубяны,None,None,None,None
2,lobanovo,лобаново,None,None,None,None,None
3,linda,линда,None,None,None,None,None
4,leskolovo,lieskula,лесколово,None,None,None,None
...,...,...,...,...,...,...,...
141116,limeira,limejra,qgb,li mei la,li mey ra,limaye'ira,лимеира
141117,itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউনা,伊塔乌纳
141118,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141119,missao,missão,None,None,None,None,None


In [ ]:
# merge to split df

In [83]:
df_world_cities_split = pd.merge(df_world_cities_split,df_alternate_names, left_index=True,right_index=True)

In [84]:
df_world_cities_split

,name,ascii_name,alternate_names,0,1,2,3,4,5,6
0,lyudinovo,lyudinovo,"ljudinovo,lyudinovo,людиново",ljudinovo,lyudinovo,людиново,None,None,None,None
1,lubyany,lubyany,"lubjany,lubyany,лубяны",lubjany,lubyany,лубяны,None,None,None,None
2,lobanovo,lobanovo,"lobanovo,лобаново",lobanovo,лобаново,None,None,None,None,None
3,linda,linda,"linda,линда",linda,линда,None,None,None,None,None
4,leskolovo,leskolovo,"leskolovo,lieskula,лесколово",leskolovo,lieskula,лесколово,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
141116,limeira,limeira,"limeira,limejra,qgb,li mei la,li mey ra,limaye...",limeira,limejra,qgb,li mei la,li mey ra,limaye'ira,лимеира
141117,itaúna,itauna,"itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউ...",itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউনা,伊塔乌纳
141118,itaporanga,itaporanga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141119,itapicuru,itapicuru,"missao,missão",missao,missão,None,None,None,None,None


In [85]:
df_world_cities_split = df_world_cities_split.drop(columns='alternate_names')

In [86]:
df_world_cities_split['ascii_name_2'] = df_world_cities_split.ascii_name

In [87]:
df_world_cities_split

,name,ascii_name,0,1,2,3,4,5,6,ascii_name_2
0,lyudinovo,lyudinovo,ljudinovo,lyudinovo,людиново,None,None,None,None,lyudinovo
1,lubyany,lubyany,lubjany,lubyany,лубяны,None,None,None,None,lubyany
2,lobanovo,lobanovo,lobanovo,лобаново,None,None,None,None,None,lobanovo
3,linda,linda,linda,линда,None,None,None,None,None,linda
4,leskolovo,leskolovo,leskolovo,lieskula,лесколово,None,None,None,None,leskolovo
...,...,...,...,...,...,...,...,...,...,...
141116,limeira,limeira,limeira,limejra,qgb,li mei la,li mey ra,limaye'ira,лимеира,limeira
141117,itaúna,itauna,itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউনা,伊塔乌纳,itauna
141118,itaporanga,itaporanga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,itaporanga
141119,itapicuru,itapicuru,missao,missão,None,None,None,None,None,itapicuru


In [94]:
list = ['city of london']
df_world_cities_split[df_world_cities_split.name.isin(list)]

,name,ascii_name,0,1,2,3,4,5,6,ascii_name_2
89442,city of london,city of london,bandaraya london,cathair londan,ceety o lunnon,cidade de londres,cite de londres,city of london,cité de londres,city of london


In [ ]:
#df_world_cities_split.name['london']# = 
#df_world_cities_split.iloc[0,'city of london']

In [119]:
dict_city_ascii_name= df_world_cities_split.set_index('ascii_name_2')['ascii_name'].to_dict()
dict_city_zero= df_world_cities_split.set_index('0')['ascii_name'].to_dict()
dict_city_one= df_world_cities_split.set_index('1')['ascii_name'].to_dict()
dict_city_two= df_world_cities_split.set_index('2')['ascii_name'].to_dict()
dict_city_three= df_world_cities_split.set_index('3')['ascii_name'].to_dict()
dict_city_four= df_world_cities_split.set_index('4')['ascii_name'].to_dict()
dict_city_five= df_world_cities_split.set_index('5')['ascii_name'].to_dict()
dict_city_six= df_world_cities_split.set_index('6')['ascii_name'].to_dict()

In [120]:
dict_city_name= df_world_cities_split.set_index('name')['ascii_name'].to_dict()

In [121]:
list_world_cities = [dict_city_zero,dict_city_ascii_name,dict_city_one,dict_city_two,dict_city_three,dict_city_four,dict_city_five,dict_city_six]

In [122]:
combined_city_dict = {key:value for x in list_world_cities for key,value in x.items() }

In [123]:
combined_city_dict

{'ljudinovo': 'lyudinovo',
 'lubjany': 'lubyany',
 'lobanovo': 'lobanovo',
 'linda': 'lindi',
 'leskolovo': 'leskolovo',
 'koesjva': 'kushva',
 'koerganinsk': 'kurganinsk',
 'krasn.tkach': 'krasnyy tkach',
 "koshekhabl'": "koshekhabl'",
 'korjazhma': 'koryazhma',
 'korobicyno': 'korobitsyno',
 'kondrova': 'kondrovo',
 'kokina': 'kokino',
 'klimov': 'klimovo',
 'kirovskij': 'balpyk bi',
 'kiricy': 'kiritsy',
 'kevsala': 'kevsala',
 'karkazskiy': 'kavkazskiy',
 'karata': 'karata',
 'izhoski': 'iznoski',
 "iban'nkoront": 'ivangorod',
 'isembai': 'ishimbay',
 'inozemcevo': 'inozemtsevo',
 'im.vorovskogo': 'imeni vorovskogo',
 'igra': 'igra',
 'goebacha': 'gubakha',
 'germenchik': 'germenchik',
 'ehtoka': 'etoka',
 'ehlektragorsk': 'elektrogorsk',
 'damadzedava': 'domodedovo',
 'dobroe': 'dobre',
 'dmitrievka': 'dmytrivka',
 'chusavoj': 'chusovoy',
 'chernukha': 'chernukha',
 'chernomorskij': 'chernomorskiy',
 'bazorkino': 'chermen',
 'batalpashinsk': 'cherkessk',
 'cajkovski': 'chaykovskiy

In [124]:
# adding edge cases 
dict_city_name['frankfurt'] = 'frankfurt am main'


In [125]:
df_user_data_cleaned['user_city'] = np.nan

In [126]:
df_user_data_cleaned.sample(20)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code,user_city
4121,87108491@N00,True,juliebugb,juliebugb,julie b,u.s.a.,2004-05-19 01:11:44,2006-06-10 12:21:10,3026,central time (us & canada),-06:00,cst6cdt,u.s.a.,0,0,NaN,NaN,NaN
3445,88195506@N00,True,boydhouse,houseofboyd,john boyd,"russell, ky., usa",2000-01-01 00:25:41,2005-09-22 01:06:33,7242,eastern time (us & canada),-05:00,est5edt,russell,ky.,usa,united states of america,us,NaN
10192,36040418@N08,True,stevejm2009,stevejm2009,steve maskell,"bournemouth, uk",1908-06-01 00:00:00,2009-03-03 13:50:38,7878,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,bournemouth,uk,0,united kingdom,gb,NaN
11838,34268979@N05,False,ross_vernal,ross_vernal (scottish dream photography),scottish dream photography,"glasgow, scotland",1931-01-01 00:00:00,2009-01-12 13:26:46,466,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,glasgow,scotland,0,scotland,gb,NaN
8499,23736722@N05,False,martinoconnor,martin e. o'connor,marty o'connor,usa,2000-01-01 00:01:35,2008-02-13 23:25:41,894,eastern time (us & canada),-05:00,est5edt,usa,0,0,united states of america,us,NaN
14177,44426956@N08,False,lemesis73,lemesis,luca,"lecce, italy",2011-01-13 23:51:15,2011-01-15 13:04:00,496,pacific time (us & canada); tijuana,-08:00,pst8pdt,lecce,italy,0,italy,it,NaN
4578,8462030@N03,True,zaretchnaia,*f~,fernanda neves,"lisboa, portugal",1840-10-01 00:00:00,2007-06-13 01:44:13,3348,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,lisboa,portugal,0,portugal,pt,NaN
5229,137506587@N06,True,darrylkirbyimages,darrylkirby,darryl kirby,"bendigo, vic, australia",2000-03-26 12:48:43,2015-11-01 03:27:03,4164,"canberra, melbourne, sydney",+10:00,australia/canberra,bendigo,vic,australia,australia,au,NaN
10615,17346825@N00,True,sharpsites,janedsh,jane holman,"richmond, united states",1861-01-01 00:00:00,2005-08-19 04:00:31,8247,indiana (east),-05:00,us/indiana-starke,richmond,united states,0,united states of america,us,NaN
4741,14161238@N08,True,darthweef,shutter bear photography,stephen day,"riverside, ca, us",2002-08-07 12:07:36,2008-02-16 23:59:51,13830,pacific time (us & canada); tijuana,-08:00,pst8pdt,riverside,ca,us,united states of america,us,NaN


In [127]:
df_user_data_cleaned.loc[12316]

user_id                                                           37357556@N00
user_ispro                                                                True
user_path_alias                                                      andreaffm
user_username                                                        andreaffm
user_realname                                                    andrea diener
user_location                                               frankfurt, germany
user_photos_firstdatetaken                                 2004-03-14 01:13:51
user_photos_firstdate                                      2005-10-18 18:56:31
user_photos_count                                                         5309
user_timezone_label           amsterdam, berlin, bern, rome, stockholm, vienna
user_timezone_offset                                                    +01:00
user_timezone_timezone_id                                     europe/amsterdam
0                                                   

In [ ]:
# for index, row in df_user_data_cleaned.iterrows():
#     x = row['0']
#     if pd.isna(row['user_city']):
#         if x in combined_city_dict:
#             df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [ ]:
combined_city_dict['frankfurt']

In [ ]:
dict_city_name['frankfurt']

## frankfurt causes trouble bc there is an alternative name of frankfort in us
so I decided to run the real name dict prior

In [128]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['0']
    if pd.isna(row['user_city']):
        if x in dict_city_name:
            df_user_data_cleaned.at[index, 'user_city']= dict_city_name[x]

In [129]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['1']
    if pd.isna(row['user_city']):
        if x in dict_city_name:
            df_user_data_cleaned.at[index, 'user_city']= dict_city_name[x]

In [130]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['2']
    if pd.isna(row['user_city']):
        if x in dict_city_name:
            df_user_data_cleaned.at[index, 'user_city']= dict_city_name[x]

In [131]:
df_user_data_cleaned.head(20)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code,user_city
0,26745338@N06,False,danisonksen,dani.sonksen,daniela sönksen,"melbourne, australia",2005-05-09 10:56:54,2008-06-05 20:03:04,850,brasilia,-03:00,america/sao_paulo,melbourne,australia,0,australia,au,melbourne
1,31331368@N00,True,lonelyradio,lonely radio,andrew j. cosgriff,"melbourne, australia",2000-02-01 00:00:00,2004-11-07 06:59:09,13381,"canberra, melbourne, sydney",+10:00,australia/canberra,melbourne,australia,0,australia,au,melbourne
2,122687277@N03,True,philipmallis,philip.mallis,philip mallis,melbourne,2007-09-21 04:37:37,2014-04-13 12:48:29,10793,<NA>,<NA>,<NA>,melbourne,0,0,NaN,NaN,melbourne
3,45008553@N04,False,reaksmeyyean,reaksmey yean – george,reaksmey yean – george,"phnom penh, kingdom of cambodia",2005-11-04 20:32:27,2009-11-27 02:43:31,236,"bangkok, hanoi, jakarta",+07:00,asia/bangkok,phnom penh,kingdom of cambodia,0,cambodia,kh,phnom penh
4,158117692@N02,True,<NA>,julie mclennan,julie mclennan,"melbourne, australia",2018-01-08 22:23:51,2018-01-09 06:23:51,2533,<NA>,<NA>,<NA>,melbourne,australia,0,australia,au,melbourne
5,169491036@N06,False,<NA>,sanzidrahmankhantamim,sanzid rahman khan tamim,"barishal, bangladesh",2015-03-24 23:04:05,2019-04-06 13:35:11,407,<NA>,<NA>,<NA>,barishal,bangladesh,0,bangladesh,bd,barishal
6,153546415@N07,True,free_aza_bird,free_aza_bird,karl baker,"melbourne, australia",2015-06-22 08:50:59,2017-04-03 01:24:16,2674,"canberra, melbourne, sydney",+10:00,australia/canberra,melbourne,australia,0,australia,au,melbourne
7,78379788@N06,True,<NA>,rob lee photography,rob lee,australia,1985-01-01 00:00:00,2014-08-20 06:35:22,2867,"canberra, melbourne, sydney",+10:00,australia/canberra,australia,0,0,australia,au,NaN
8,115521748@N08,False,<NA>,footeefok,tee fok foo,"singapore, singapore",2008-08-24 12:38:45,2014-01-25 15:18:44,20678,"kuala lumpur, singapore",+08:00,asia/kuala_lumpur,singapore,singapore,0,singapore,sg,singapore
9,44340545@N05,True,gmpolice1,greater manchester police,greater manchester police,"manchester, greater manchester",1840-01-01 00:00:00,2009-11-05 13:53:07,27764,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,manchester,greater manchester,0,NaN,NaN,manchester


In [138]:
df_user_data_cleaned.user_city.isna().sum()

2827

In [133]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['0']
    if pd.isna(row['user_city']):
        if x in combined_city_dict:
            df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [135]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['1']
    if pd.isna(row['user_city']):
        if x in combined_city_dict:
            df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [137]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['2']
    if pd.isna(row['user_city']):
        if x in combined_city_dict:
            df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [139]:
df_user_data_cleaned.sample(15)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code,user_city
7295,39123712@N03,True,monfolio,monfolio (chr. bernard),chr. bernard,bruxelles,2007-08-29 11:10:47,2009-06-11 10:48:54,2747,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,bruxelles,0,0,NaN,NaN,NaN
3193,39980275@N05,True,<NA>,andrew page1,andrew page,"cardiff, wales",NaT,2009-07-03 01:10:15,2431,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,cardiff,wales,0,wales,gb,cardiff
1032,61819511@N02,True,shaunsgrimepics,shaun the grime lover,shaun,united kingdom,2007-10-02 14:10:35,2011-04-16 15:45:00,4460,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,united kingdom,0,0,united kingdom,gb,NaN
10023,78427569@N07,True,umbralx,umbralx,mr. shadow,"south florida, usa",2009-02-01 00:00:00,2012-03-28 02:48:25,2187,eastern time (us & canada),-05:00,est5edt,south florida,usa,0,united states of america,us,usa
8861,193730554@N03,False,<NA>,mayer20204,dieter mayer,hückeswagen,2014-10-28 20:19:45,2021-08-22 09:17:13,63,<NA>,<NA>,<NA>,hückeswagen,0,0,NaN,NaN,hueckeswagen
1339,149930267@N08,True,davejforjohnstone,dave the bluesman.,dave johnstone,"peebles, scotland, scotland",2015-03-10 12:59:23,2019-03-09 14:10:39,3799,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,peebles,scotland,scotland,scotland,gb,peebles
1620,192961344@N05,False,<NA>,kalingastone,kalinga stone,"mumbai, india",NaT,2021-05-12 09:55:22,146,pacific time (us & canada); tijuana,-08:00,pst8pdt,mumbai,india,0,india,in,mumbai
6356,60102727@N00,False,lisa_simpsonlk,lisa_simpsonlyke,elisa ferrari,"córdoba, argentina",2005-01-01 00:00:50,2012-11-30 16:50:11,1577,"buenos aires, georgetown",-03:00,america/buenos_aires,córdoba,argentina,0,argentina,ar,cordoba
7911,94191024@N06,False,uru-guru,uru-guru,<NA>,"montevideo, uruguay",2009-07-13 12:25:14,2013-03-18 20:22:30,520,"buenos aires, georgetown",-03:00,america/buenos_aires,montevideo,uruguay,0,uruguay,uy,montevideo
1288,26706225@N08,True,hugo_poon,hugo p - one day in my life,hugo p,hong kong,2005-04-17 21:23:10,2008-05-17 04:01:35,4773,"beijing, chongqing, hong kong, urumqi",+08:00,asia/chongqing,hong kong,0,0,hong kong,hk,hong kong


In [140]:
df_user_data_cleaned.tail(20)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code,user_city
14449,46073170@N05,True,flow_of_pictures,kptnflow,florian fievet,"montréal, canada",1990-01-01 00:00:00,2010-01-02 16:13:29,5002,eastern time (us & canada),-05:00,est5edt,montréal,canada,0,canada,ca,montreal
14450,191523917@N07,True,huub-janssen,huub.janssen,huub janssen,"venlo-blerick, nederland",2020-11-24 06:15:08,2020-12-24 10:37:27,3336,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,venlo-blerick,nederland,0,netherlands,nl,nederland
14451,44372192@N00,True,matisok,matisok,mathias willerup,"copenhagen, denmark",1983-07-01 00:00:00,2000-05-22 21:14:44,4048,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,copenhagen,denmark,0,denmark,dk,copenhagen
14452,18839565@N00,False,daniel_pettinger,daniel pettinger,daniel pettinger,"linz, austria",2004-03-29 19:48:16,2006-01-15 16:31:41,10564,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,linz,austria,0,austria,at,linz
14453,68574290@N00,False,julien-falgas,webcomics.fr,<NA>,france,2007-06-03 22:24:11,2007-06-03 20:24:11,53,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,france,0,0,france,fr,NaN
14454,24642077@N02,False,efrensl,efrén sl,efrén sánchez,españa,2010-03-13 17:38:41,2010-03-25 22:37:12,347,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,españa,0,0,spain,es,espana
14455,65059720@N06,False,<NA>,marc sullivan,marc sullivan,"hamburg, deutschland",2013-06-26 17:54:37,2019-03-22 21:33:41,146,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,hamburg,deutschland,0,germany,de,hamburg
14456,95482784@N00,False,robaer,robert fritz,robert fritz,austria,2005-10-02 14:14:12,2006-05-08 11:02:33,439,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,austria,0,0,austria,at,NaN
14457,191808253@N06,False,champland,champland,au fil des routes et des chemins de.... ...,"marne, france",1981-01-01 00:00:00,2021-01-18 22:20:56,863,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,marne,france,0,france,fr,NaN
14458,51485383@N02,False,sergioloca,agoralex,sergio locatelli,"casorate primo (pv), italia",2009-05-01 17:50:43,2010-07-03 22:00:37,939,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,casorate primo (pv),italia,0,italy,it,NaN


In [ ]:
## check edge cases

In [ ]:
df_user_data_cleaned.loc[9958]

In [ ]:
df_user_data_cleaned.loc[5935]

In [ ]:
df_user_data_cleaned.loc[8]

In [ ]:
df_user_data_cleaned.shape

In [141]:
df_user_data_cleaned.user_city.isna().sum()

2827

In [142]:
df_user_data_cleaned.user_country.isna().sum()

2398

In [143]:
df_world_cities.sample(7)

,geoname_id,name,ascii_name,alternate_names,feature_class,feature_code,country_code,country_name_en,country_code_2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,digital_elevation_model,timezone,modification_date,label_en,coordinates
44960,2653915,Campbeltown,Campbeltown,"Balley Keeill Charane,CAL,Cammeltoun,Campbelto...",P,PPL,GB,United Kingdom,NaN,SCT,T8,NaN,NaN,4800,NaN,5,Europe/London,2017-06-12,United Kingdom,"55.42583, -5.60764"
136748,4002574,La Encarnación,La Encarnacion,"Encarnacion,Encarnación,La Encarnacion,La Enca...",P,PPL,MX,Mexico,NaN,16,112,NaN,NaN,1714,NaN,1733,America/Mexico_City,2018-11-03,Mexico,"19.41242, -100.39211"
2872,3178620,Città della Pieve,Citta della Pieve,"Chita dela Pieve,Citta della Pieve,Città della...",P,PPLA3,IT,Italy,NaN,18,PG,054012,NaN,3360,509.0,484,Europe/Rome,2019-10-10,Italy,"42.95934, 12.00696"
51665,3054352,Császártöltés,Csaszartoltes,"Csaszartoltes,Császártöltés,Kaiserdamm,Tetis,T...",P,PPL,HU,Hungary,NaN,01,NaN,NaN,NaN,2649,NaN,109,Europe/Budapest,2021-07-15,Hungary,"46.42194, 19.18361"
5131,2016308,Solnechnyy,Solnechnyy,"Solnechnyj,Solnechnyy,Solnetsjnyj,Солнечный",P,PPL,RU,Russian Federation,NaN,30,NaN,NaN,NaN,13993,NaN,33,Asia/Vladivostok,2012-01-17,Russian Federation,"50.58333, 137.03333"
16022,1058080,Nosy Varika,Nosy Varika,NaN,P,PPL,MG,Madagascar,NaN,27,23207,23207010,23207010001,37152,NaN,8,Indian/Antananarivo,2022-09-30,Madagascar,"-20.58333, 48.53333"
92147,3980910,Tumbiscatío de Ruiz,Tumbiscatio de Ruiz,"Tumbiscatio,Tumbiscatio de Ruiz,Tumbiscatío de...",P,PPLA2,MX,Mexico,NaN,16,096,NaN,NaN,2801,NaN,919,America/Mexico_City,2020-06-10,Mexico,"18.52667, -102.37823"


In [144]:
world_city_columns_keep = ['ascii_name','country_name_en','country_code','population','timezone','coordinates']

In [145]:
df_world_cities = df_world_cities.loc[:,world_city_columns_keep]

In [146]:
# add prefix except ascii_name
df_world_cities = df_world_cities.add_prefix('city_')


In [ ]:
#df_world_cities['user_city'] = df_world_cities['city_ascii_name']

In [147]:
df_world_cities.sample(5)

,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates
26633,Montano Lucino,Italy,IT,4296,Europe/Rome,"45.78378, 9.04244"
118368,Cervignano d'Adda,Italy,IT,2069,Europe/Rome,"45.37387, 9.42405"
30471,Septfonds,France,FR,1989,Europe/Paris,"44.17831, 1.61806"
122514,Mount Ayr,United States,US,1686,America/Chicago,"40.71471, -94.23523"
18361,Barssel,Germany,DE,12475,Europe/Berlin,"53.16981, 7.75012"


In [ ]:
#df_world_cities[df_world_cities.city_country_name_en=="Korea, Republic of"]

In [ ]:
# df_world_countries.head()
# df_world_countries[df_world_countries.alpha2=='cz']

In [ ]:
#df_world_countries['Czech Republic']='Czechia'

In [ ]:
#df_user_data_cleaned_city = df_user_data_cleaned.merge(df_world_cities, left_on='user_city',right_on='city_ascii_name',how='left')

In [ ]:
#df_user_data_cleaned_country=df_user_data_cleaned.merge(df_world_cities, left_on='user_country',right_on='city_country_name_en',how='left')

In [ ]:
#df_user_data_cleaned_country.head(20)

In [148]:
df_world_cities.city_ascii_name = df_world_cities.city_ascii_name.str.lower()

In [151]:
df_world_cities.city_country_name_en = df_world_cities.city_country_name_en.str.lower()
df_user_data_cleaned.user_country = df_user_data_cleaned.user_country.str.lower()
df_world_cities.city_timezone = df_world_cities.city_timezone.str.lower()
df_world_cities.city_country_code = df_world_cities.city_country_code.str.lower()

In [153]:
df_world_cities = df_world_cities.apply(lambda x: x.astype('string').str.strip() if x.dtype == "object" else x) 

In [154]:
df_world_cities.head()

,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates
0,lyudinovo,russian federation,ru,41392,europe/moscow,"53.86639, 34.44778"
1,lubyany,russian federation,ru,1899,europe/moscow,"56.0378, 51.40001"
2,lobanovo,russian federation,ru,3563,asia/yekaterinburg,"57.8595, 56.3026"
3,linda,russian federation,ru,5489,europe/moscow,"56.61571, 44.09544"
4,leskolovo,russian federation,ru,4039,europe/moscow,"60.26341, 30.45462"


In [160]:
df_world_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141121 entries, 0 to 141120
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   city_ascii_name       141120 non-null  string
 1   city_country_name_en  140953 non-null  string
 2   city_country_code     141076 non-null  string
 3   city_population       141121 non-null  int64 
 4   city_timezone         141121 non-null  string
 5   city_coordinates      141121 non-null  string
 6   city_lat              141121 non-null  string
 7   city_long             141121 non-null  string
dtypes: int64(1), string(7)
memory usage: 8.6 MB


In [156]:
# split coordinates and merge to df world cities
df_c_split = df_world_cities.city_coordinates.str.split(', ', expand=True)
df_c_split.columns = df_c_split.columns.astype('string')
df_c_split = df_c_split.rename(columns={'0':'city_lat','1':'city_long'})

In [157]:
df_c_split.head()

,city_lat,city_long
0,53.86639,34.44778
1,56.0378,51.40001
2,57.8595,56.3026
3,56.61571,44.09544
4,60.26341,30.45462


In [158]:
df_world_cities = df_world_cities.merge(df_c_split, left_index=True,right_index=True,how='left')

In [159]:
df_world_cities.head()

,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates,city_lat,city_long
0,lyudinovo,russian federation,ru,41392,europe/moscow,"53.86639, 34.44778",53.86639,34.44778
1,lubyany,russian federation,ru,1899,europe/moscow,"56.0378, 51.40001",56.0378,51.40001
2,lobanovo,russian federation,ru,3563,asia/yekaterinburg,"57.8595, 56.3026",57.8595,56.3026
3,linda,russian federation,ru,5489,europe/moscow,"56.61571, 44.09544",56.61571,44.09544
4,leskolovo,russian federation,ru,4039,europe/moscow,"60.26341, 30.45462",60.26341,30.45462


In [161]:
df_user_data_cleaned_final=df_user_data_cleaned.merge(df_world_cities, left_on=['user_country_code','user_city'],right_on=['city_country_code','city_ascii_name'],how='left')

In [162]:
df_user_data_cleaned_final.head(20)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,...,user_country_code,user_city,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates,city_lat,city_long
0,26745338@N06,False,danisonksen,dani.sonksen,daniela sönksen,"melbourne, australia",2005-05-09 10:56:54,2008-06-05 20:03:04,850,brasilia,...,au,melbourne,melbourne,australia,au,4917750.0,australia/melbourne,"-37.814, 144.96332",-37.814,144.96332
1,31331368@N00,True,lonelyradio,lonely radio,andrew j. cosgriff,"melbourne, australia",2000-02-01 00:00:00,2004-11-07 06:59:09,13381,"canberra, melbourne, sydney",...,au,melbourne,melbourne,australia,au,4917750.0,australia/melbourne,"-37.814, 144.96332",-37.814,144.96332
2,122687277@N03,True,philipmallis,philip.mallis,philip mallis,melbourne,2007-09-21 04:37:37,2014-04-13 12:48:29,10793,<NA>,...,NaN,melbourne,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
3,45008553@N04,False,reaksmeyyean,reaksmey yean – george,reaksmey yean – george,"phnom penh, kingdom of cambodia",2005-11-04 20:32:27,2009-11-27 02:43:31,236,"bangkok, hanoi, jakarta",...,kh,phnom penh,phnom penh,cambodia,kh,1573544.0,asia/phnom_penh,"11.56245, 104.91601",11.56245,104.91601
4,158117692@N02,True,<NA>,julie mclennan,julie mclennan,"melbourne, australia",2018-01-08 22:23:51,2018-01-09 06:23:51,2533,<NA>,...,au,melbourne,melbourne,australia,au,4917750.0,australia/melbourne,"-37.814, 144.96332",-37.814,144.96332
5,169491036@N06,False,<NA>,sanzidrahmankhantamim,sanzid rahman khan tamim,"barishal, bangladesh",2015-03-24 23:04:05,2019-04-06 13:35:11,407,<NA>,...,bd,barishal,barishal,bangladesh,bd,202242.0,asia/dhaka,"22.70497, 90.37013",22.70497,90.37013
6,153546415@N07,True,free_aza_bird,free_aza_bird,karl baker,"melbourne, australia",2015-06-22 08:50:59,2017-04-03 01:24:16,2674,"canberra, melbourne, sydney",...,au,melbourne,melbourne,australia,au,4917750.0,australia/melbourne,"-37.814, 144.96332",-37.814,144.96332
7,78379788@N06,True,<NA>,rob lee photography,rob lee,australia,1985-01-01 00:00:00,2014-08-20 06:35:22,2867,"canberra, melbourne, sydney",...,au,NaN,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
8,115521748@N08,False,<NA>,footeefok,tee fok foo,"singapore, singapore",2008-08-24 12:38:45,2014-01-25 15:18:44,20678,"kuala lumpur, singapore",...,sg,singapore,singapore,singapore,sg,5638700.0,asia/singapore,"1.28967, 103.85007",1.28967,103.85007
9,44340545@N05,True,gmpolice1,greater manchester police,greater manchester police,"manchester, greater manchester",1840-01-01 00:00:00,2009-11-05 13:53:07,27764,"gmt: dublin, edinburgh, lisbon, london",...,NaN,manchester,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>


In [163]:
df_user_data_cleaned_final.loc[10]

user_id                                                 45676495@N05
user_ispro                                                      True
user_path_alias                                                 <NA>
user_username                                         robertknight16
user_realname                                          robert knight
user_location                                        colton, england
user_photos_firstdatetaken                       2000-01-01 00:00:00
user_photos_firstdate                            2009-12-23 17:44:08
user_photos_count                                              42784
user_timezone_label           gmt: dublin, edinburgh, lisbon, london
user_timezone_offset                                          +00:00
user_timezone_timezone_id                              europe/london
0                                                             colton
1                                                            england
2                                 

In [173]:
df_user_data_cleaned_final[df_user_data_cleaned_final.city_ascii_name=='london']

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,...,user_country_code,user_city,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates,city_lat,city_long
100,90538631@N03,False,arnabkghosal,arnabkghosal,arnab ghosal,"london, united kingdom",2012-08-12 13:24:13,2013-01-13 18:41:36,672,"gmt: dublin, edinburgh, lisbon, london",...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
123,187325247@N07,False,<NA>,ian ware's transport photography gallery,<NA>,"london, england",2019-08-31 15:08:19,2020-03-07 19:28:35,534,"gmt: dublin, edinburgh, lisbon, london",...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
131,123951360@N03,True,<NA>,bill_giddings,bill giddings,"london, uk",2004-06-17 13:50:36,2014-05-16 17:36:11,457,"gmt: dublin, edinburgh, lisbon, london",...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
210,33909206@N04,True,geezaweezer,geraint rowland photography,geraint rowland,"london, united kingdom",2009-01-04 22:47:07,2009-08-26 11:17:10,5407,"gmt: dublin, edinburgh, lisbon, london",...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
219,34427470616@N01,True,duncan,duncan cumming,duncan cumming,"london, united kingdom",1997-01-01 00:02:00,2004-02-06 23:30:28,145404,"gmt: dublin, edinburgh, lisbon, london",...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19405,54517219@N08,False,simonr0,simon-radford,simon radford,"london, uk",2001-06-03 09:00:00,2010-10-01 23:00:00,385,"gmt: dublin, edinburgh, lisbon, london",...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
19456,31779113@N06,True,<NA>,andrew g robertson,andrew robertson,"london, united kingdom",2008-07-23 12:42:04,2009-12-24 07:53:03,1483,"gmt: dublin, edinburgh, lisbon, london",...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
19490,88077847@N07,True,andreaheribanova,andrea heribanova,andrea heribanova,"london, uk",2012-10-30 14:27:35,2013-05-04 11:25:44,161,"gmt: dublin, edinburgh, lisbon, london",...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
19600,98119031@N00,True,dick_pountain,dick_pountain,dick pountain,"london, uk",2000-04-25 22:06:02,2006-04-03 22:00:00,2102,"gmt: dublin, edinburgh, lisbon, london",...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574


In [174]:
df_user_data_cleaned_final.shape

(19858, 26)

In [190]:
df_user_data_cleaned_final.city_coordinates.isna().sum()

0

In [ ]:
#df_user_data_cleaned_city.shape

In [ ]:
#df_user_data_cleaned_country.shape

In [176]:
df_user_data_cleaned_final.sample(15)

,user_id,user_ispro,user_path_alias,user_username,user_realname,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,...,user_country_code,user_city,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates,city_lat,city_long
11179,98877009@N02,True,dougcraig1,exploring our world,doug craig travel photography,"busan, south korea",2004-12-23 07:09:17,2013-07-15 14:09:29,369,seoul,...,NaN,busan,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
2456,7798484@N05,False,littlelioness09,little lioness,sarah bartell,"portland, united states",2003-01-01 16:59:53,2007-04-30 03:52:01,978,pacific time (us & canada); tijuana,...,us,portland,portland,united states,us,3923.0,america/detroit,"42.8692, -84.90305",42.8692,-84.90305
14126,156812453@N07,False,ricosz2,bugs 2019-2023,richard,"lukow (łuków), poland",2019-03-22 14:00:00,2019-03-22 20:58:31,440,"sarajevo, skopje, warsaw, zagreb",...,pl,poland,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
15736,13363765@N00,True,vapspwi,vapspwi,jerry ray,"fayetteville, ga, usa",2004-12-21 00:00:00,2005-07-08 02:26:25,6962,eastern time (us & canada),...,us,fayetteville,fayetteville,united states,us,7121.0,america/chicago,"35.15203, -86.57055",35.15203,-86.57055
18750,142289664@N02,True,<NA>,richardcorkish,richard corkish,sydney,NaT,2016-04-27 12:18:07,20726,<NA>,...,NaN,sydney,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
11941,62588967@N03,True,jonpsphotos,jonpspics,jonathan payne,"ayr, united kingdom",1978-01-01 00:00:00,2011-05-07 12:43:46,4395,"gmt: dublin, edinburgh, lisbon, london",...,gb,ayr,ayr,united kingdom,gb,46260.0,europe/london,"55.46273, -4.63393",55.46273,-4.63393
14162,31018257@N00,False,3336,diego3336,diego torres silvestre,"sao paulo, brazil",1999-01-02 18:42:35,2005-03-10 12:31:04,1502,"buenos aires, georgetown",...,br,brazil,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
14744,95883232@N07,False,rabican,rabican7,<NA>,"manchester, nh, usa",2010-09-26 19:05:56,2013-05-20 10:32:32,643,eastern time (us & canada),...,us,manchester,manchester,united states,us,18229.0,america/chicago,"38.597, -90.50929",38.597,-90.50929
14125,11231432@N03,True,prankf,prank f,frank p,"northants, uk",2001-06-12 11:20:54,2007-08-10 17:34:25,24424,"gmt: dublin, edinburgh, lisbon, london",...,gb,uk,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
18741,58628800@N05,True,infinity-and-beyond,infinity & beyond photography: kev cook,kev cook,"miami, usa",1982-05-01 00:00:00,2011-01-23 05:43:31,8181,eastern time (us & canada),...,us,miami,miami,united states,us,589.0,america/chicago,"35.69143, -100.63819",35.69143,-100.63819


In [ ]:
# compare countries
#df_user_data_cleaned['equal_country'] = df_user_data_cleaned['user_country'].eq(df_user_data_cleaned['city_country_name_en'])

In [ ]:
df_user_data_cleaned_final.head(30)

In [ ]:
# keep false + NaN
#df_user_data_cleaned[df_user_data_cleaned['equal_country']== True]

In [177]:
df_user_data_cleaned_final.city_coordinates.count()

13342

In [181]:
df_user_data_cleaned_final[df_user_data_cleaned_final[user_country]]

NameError: name 'user_country' is not defined

In [182]:
df_user_data_cleaned_final.shape


(19858, 26)

In [185]:
df_user_data_cleaned_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19858 entries, 0 to 19857
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   user_id                     19858 non-null  string        
 1   user_ispro                  19858 non-null  bool          
 2   user_path_alias             14107 non-null  string        
 3   user_username               19858 non-null  string        
 4   user_realname               17380 non-null  string        
 5   user_location               19858 non-null  string        
 6   user_photos_firstdatetaken  18756 non-null  datetime64[ns]
 7   user_photos_firstdate       19857 non-null  datetime64[ns]
 8   user_photos_count           19858 non-null  int64         
 9   user_timezone_label         18371 non-null  string        
 10  user_timezone_offset        18371 non-null  string        
 11  user_timezone_timezone_id   18371 non-null  string    

In [184]:
columns_to_drop_final = ['0','1','2']
df_user_data_cleaned_final = df_user_data_cleaned_final.drop(columns=columns_to_drop_final)

In [186]:
df_user_data_cleaned_final = df_user_data_cleaned_final.drop_duplicates(subset='user_id')

In [187]:
df_user_data_cleaned_final.shape

(14172, 23)

In [188]:
df_user_data_cleaned_final = df_user_data_cleaned_final.dropna(subset='city_coordinates')

In [189]:
df_user_data_cleaned_final.shape

(7656, 23)

In [192]:
df_user_data_cleaned_final.to_csv('data/clean_user.csv', sep=',')

In [196]:
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests
from sql_functions import get_engine

In [201]:
table_name = 'clean_user'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...
engine = get_engine()
schema = 'capstone_jorittega'
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_user_data_cleaned_final.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The clean_user table was imported successfully.
